Импорт и настройки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Настройки отображения
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.grid"] = True


In [ ]:
from pathlib import Path

# Пути к файлам для US-модели
US_FORECAST_PATH = "input_US.xlsb"
US_FORECAST_SHEET = "pivot"   # если лист называется иначе — поменяй здесь

US_CATALOG_PATH = "catalog_US.xlsb"
US_CATALOG_SHEET = "Sheet1"

# Чтение прогноза из xlsb (нужен pyxlsb: pip install pyxlsb)
df_us_raw = pd.read_excel(
    US_FORECAST_PATH,
    sheet_name=US_FORECAST_SHEET,
    engine="pyxlsb"
)

df_us_raw.head()


Очистка

In [ ]:
df_us = df_us_raw.copy()

# Нормализуем названия колонок (на случай разных регистров)
df_us.columns = [str(c).strip() for c in df_us.columns]

# Оставляем только нужные поля
keep_cols = ["item_id", "timestamp", "mean"]
df_us = df_us[[c for c in df_us.columns if c in keep_cols]]

df_us.head()


In [ ]:
# Убедимся, что item_id — строка
df_us["item_id"] = df_us["item_id"].astype(str)

# Формат: <driver_code>+<subdivision_code>
item_parts_us = df_us["item_id"].str.split("+", expand=True)

df_us["driver_code"] = item_parts_us[0].str.strip()
df_us["subdivision_code"] = item_parts_us[1].str.strip()

df_us.head()


In [ ]:
# timestamp как в первой модели: YYYY-MM-DD
df_us["timestamp"] = pd.to_datetime(df_us["timestamp"], format="%Y-%m-%d")

df_us["year"] = df_us["timestamp"].dt.year
df_us["quarter_num"] = df_us["timestamp"].dt.quarter
df_us["quarter"] = df_us["quarter_num"].astype(str) + "Q"
df_us["month"] = df_us["timestamp"].dt.month

df_us.head()


In [ ]:
df_us["mean"] = pd.to_numeric(df_us["mean"], errors="coerce")

df_us["mean"].describe()


Подготовка справочника

In [ ]:
df_us_catalog_raw = pd.read_excel(
    US_CATALOG_PATH,
    sheet_name=US_CATALOG_SHEET,
    engine="pyxlsb"
)

df_us_catalog_raw.head()


In [ ]:
df_us_catalog = df_us_catalog_raw.copy()

# Приведём названия к удобным
# Будем считать:
#   колонка C -> category
#   колонка D -> driver_code (совпадает с driver_code в прогнозе)
col_c = df_us_catalog.columns[2]   # C
col_d = df_us_catalog.columns[3]   # D

df_us_catalog = df_us_catalog.rename(columns={
    col_c: "category",
    col_d: "driver_code"
})

# Оставим только нужные столбцы и удалим дубликаты
df_us_catalog = df_us_catalog[["driver_code", "category"]].copy()

df_us_catalog["driver_code"] = df_us_catalog["driver_code"].astype(str).str.strip()
df_us_catalog["category"] = df_us_catalog["category"].astype(str).str.strip()

df_us_catalog = df_us_catalog.drop_duplicates()

df_us_catalog.head()


In [ ]:
df_us["driver_code"] = df_us["driver_code"].astype(str).str.strip()

df_us_enriched = df_us.merge(
    df_us_catalog,
    how="left",
    on="driver_code"
)

df_us_enriched.head()


In [ ]:
print("Всего строк:", len(df_us_enriched))
print("С ненайденной категорией (category is NA):",
      df_us_enriched["category"].isna().sum())

df_us_enriched["category"].value_counts(dropna=False).head(20)


Агрегирование: сумма mean по категории и месяцу

In [ ]:
us_agg_cat_month = (
    df_us_enriched
    .groupby(["year", "month", "category"], as_index=False)
    .agg(total_mean=("mean", "sum"))
)

us_agg_cat_month.head()


WAPE

In [ ]:
df_fact_us_raw = pd.read_excel(
    US_FACT_PATH,
    sheet_name=US_FACT_SHEET,
    engine="pyxlsb"
)

df_fact_us_raw.head()

df_fact_us = df_fact_us_raw.copy()

# Переименуем по индексам колонок
cols = df_fact_us.columns

fact_us_rename = {
    cols[0]:  "subdivision_code",  # A
    cols[1]:  "subdivision_text",  # B
    # пропускаем C..L, если есть
    cols[12]: "driver_code",       # M
    cols[13]: "driver_text",       # N
    cols[14]: "year",              # O
    cols[15]: "month",             # P
    cols[16]: "fact"               # Q
}

df_fact_us = df_fact_us.rename(columns=fact_us_rename)



# Приводим типы
df_fact_us["subdivision_code"] = df_fact_us["subdivision_code"].astype(str).str.strip()
df_fact_us["driver_code"] = df_fact_us["driver_code"].astype(str).str.strip()

df_fact_us["year"] = pd.to_numeric(df_fact_us["year"], errors="coerce").astype("Int64")
df_fact_us["month"] = pd.to_numeric(df_fact_us["month"], errors="coerce").astype("Int64")
df_fact_us["fact"] = pd.to_numeric(df_fact_us["fact"], errors="coerce")

df_fact_us.head()



df_fact_us_period = df_fact_us[
    (df_fact_us["year"] == WAPE_YEAR_US) &
    (df_fact_us["month"].isin(WAPE_MONTHS_US))
].copy()

df_fact_us_period.head()

df_us_forecast_period = df_us_enriched[
    (df_us_enriched["year"] == WAPE_YEAR_US) &
    (df_us_enriched["month"].isin(WAPE_MONTHS_US))
].copy()

df_us_forecast_period.head()



#Слияние прогноза и факта + расчёт абсолютной ошибки
merge_cols_us = ["driver_code", "subdivision_code", "year", "month"]

df_us_forecast_period["mean_forecast"] = df_us_forecast_period["mean"]

cols_forecast_merge_us = merge_cols_us + ["mean_forecast", "category"]

df_us_to_merge = df_us_forecast_period[cols_forecast_merge_us].copy()

df_us_merged = df_us_to_merge.merge(
    df_fact_us_period[merge_cols_us + ["fact", "subdivision_text", "driver_text"]],
    how="inner",
    on=merge_cols_us
)

df_us_merged["abs_error"] = (df_us_merged["mean_forecast"] - df_us_merged["fact"]).abs()

df_us_merged.head()



#Расчёт WAPE в разрезе драйвер / подразделение / категория
group_cols_us = [
    "driver_code", "driver_text",
    "subdivision_code", "subdivision_text",
    "category"
]

wape_us_result = (
    df_us_merged
    .groupby(group_cols_us, as_index=False)
    .agg(
        abs_error_sum=("abs_error", "sum"),
        fact_sum=("fact", "sum")
    )
)

wape_us_result["wape"] = wape_us_result["abs_error_sum"] / wape_us_result["fact_sum"]
wape_us_result["wape_pct"] = wape_us_result["wape"] * 100

wape_us_result.sort_values("wape", ascending=False).head(20)

График: TOP-N драйверов по WAPE (US)

In [ ]:
top_n_us = 20

wape_us_by_driver = (
    wape_us_result
    .groupby(["driver_code", "driver_text", "category"], as_index=False)
    .agg(
        abs_error_sum=("abs_error_sum", "sum"),
        fact_sum=("fact_sum", "sum")
    )
)
wape_us_by_driver["wape"] = wape_us_by_driver["abs_error_sum"] / wape_us_by_driver["fact_sum"]
wape_us_by_driver["wape_pct"] = wape_us_by_driver["wape"] * 100

wape_us_top = wape_us_by_driver.sort_values("wape", ascending=False).head(top_n_us)

plt.figure()
plt.bar(
    wape_us_top["driver_text"].fillna(wape_us_top["driver_code"]),
    wape_us_top["wape_pct"]
)
plt.xticks(rotation=60, ha="right")
plt.ylabel("WAPE, %")
plt.title(f"US: TOP-{top_n_us} драйверов по WAPE ({WAPE_YEAR_US}, месяцы {WAPE_MONTHS_US[0]}–{WAPE_MONTHS_US[-1]})")
plt.tight_layout()
plt.show()

wape_us_top.head()


График: прогноз vs факт по одному драйверу

In [ ]:
# Берём "самый плохой" драйвер из топа
example_us_driver = wape_us_top.iloc[0]["driver_code"]

df_us_example = df_us_merged[df_us_merged["driver_code"] == example_us_driver].copy()
df_us_example = df_us_example.sort_values("month")

plt.figure()
plt.plot(df_us_example["month"], df_us_example["mean_forecast"], marker="o", label="Прогноз (mean)")
plt.plot(df_us_example["month"], df_us_example["fact"], marker="o", label="Факт")
plt.xticks(df_us_example["month"].unique())
plt.xlabel("Месяц")
plt.ylabel("Значение")
plt.title(f"US: Прогноз vs Факт для драйвера {example_us_driver}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

df_us_example[["year", "month", "subdivision_code", "driver_code", "category", "mean_forecast", "fact"]].head()


График: динамика по категориям (по месяцам)

In [ ]:
# Возьмём один год (например, самый свежий)
latest_year_us = us_agg_cat_month["year"].max()
df_plot = us_agg_cat_month[us_agg_cat_month["year"] == latest_year_us].copy()

plt.figure()

for cat, grp in df_plot.groupby("category"):
    plt.plot(grp["month"], grp["total_mean"], marker="o", label=str(cat))

plt.xlabel("Месяц")
plt.ylabel("Сумма mean")
plt.title(f"Сумма mean по категориям, {latest_year_us} год")
plt.xticks(sorted(df_plot["month"].unique()))
plt.legend(title="Категория", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


График: барчарт по категориям за последний месяц

In [ ]:
latest_year_us = df_us_enriched["year"].max()
latest_month_us = df_us_enriched[df_us_enriched["year"] == latest_year_us]["month"].max()

df_last = (
    df_us_enriched[
        (df_us_enriched["year"] == latest_year_us) &
        (df_us_enriched["month"] == latest_month_us)
    ]
    .groupby("category", as_index=False)
    .agg(total_mean=("mean", "sum"))
    .sort_values("total_mean", ascending=False)
)

plt.figure()
plt.bar(df_last["category"], df_last["total_mean"])
plt.xticks(rotation=60, ha="right")
plt.xlabel("Категория")
plt.ylabel("Сумма mean")
plt.title(f"Сумма mean по категориям, {latest_year_us}-{latest_month_us:02d}")
plt.tight_layout()
plt.show()

df_last.head()


Фильтры по категории / подразделению

In [ ]:
US_FILTER_CATEGORIES = []          # пример: ["Revenue", "Cost"]
US_FILTER_SUBDIVISION_CODES = []   # пример: ["US001", "US002"]

def us_apply_filters(df):
    df_f = df.copy()
    
    if US_FILTER_CATEGORIES:
        df_f = df_f[df_f["category"].isin(US_FILTER_CATEGORIES)]
    if US_FILTER_SUBDIVISION_CODES:
        df_f = df_f[df_f["subdivision_code"].isin(US_FILTER_SUBDIVISION_CODES)]
    
    return df_f

df_us_filtered = us_apply_filters(df_us_enriched)
df_us_filtered.head()


Сводная таблица: категория × месяц

In [ ]:
# Сводная по последнему году
latest_year_us = df_us_enriched["year"].max()

df_pivot = df_us_enriched[df_us_enriched["year"] == latest_year_us].pivot_table(
    index="category",
    columns="month",
    values="mean",
    aggfunc="sum",
    fill_value=0
)

df_pivot
